### Read in the libraries

In [14]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

### Read in the data

In [15]:
df_combined = pd.read_csv('../data/df_combined_trading.csv')

### Define current weights

In [16]:
current_weights = {
    'SPY': 0.8054,
    'TLT': 0.0000,
    'BTC': 0.1666,
    'CASH': 0.0280
}

### Set up the parameters and calculate signals

In [30]:
# ==========================
# Initial Parameters Setup (Modified)
# ==========================
initial_investment = 100000  # Starting portfolio value in USD
commission_rate = 0.001  # 0.1% commission per trade

# Copy dataframe to prevent modification of original data
trading_results = df_combined.copy().reset_index(drop=True)

# ==============================
# Initialize Market Prices
# ==============================
initial_btc_price = trading_results['BTC_Actual'].iloc[0]
initial_spy_price = trading_results['SPY_Actual'].iloc[0]
initial_tlt_price = trading_results['TLT_Close'].iloc[0]

# Flag when SPY and TLT markets are closed
trading_results['SPY_Closed'] = trading_results['SPY_Actual'].isna()
trading_results['TLT_Closed'] = trading_results['TLT_Close'].isna()

# ============================================================
# Compute Rolling Volatility (Risk Estimation)
# ============================================================
trading_results['BTC_Volatility'] = trading_results['BTC_Actual'].pct_change().rolling(1440).std()
trading_results['SPY_Volatility'] = trading_results['SPY_Actual'].pct_change().rolling(1440).std()

# Fill missing volatility with the first valid value (or a small constant)
trading_results['BTC_Volatility'].fillna(method='bfill', inplace=True)
trading_results['SPY_Volatility'].fillna(method='bfill', inplace=True)

# ============================================================
# Compute Bollinger Bands for BTC and SPY
# ============================================================
def calculate_bollinger_bands(df, column, window=20, num_std=2):
    """Calculate Bollinger Bands."""
    rolling_mean = df[column].rolling(window=window).mean()
    rolling_std = df[column].rolling(window=window).std()
    upper_band = rolling_mean + (num_std * rolling_std)
    lower_band = rolling_mean - (num_std * rolling_std)
    return rolling_mean, upper_band, lower_band

trading_results['BTC_BB_Mid'], trading_results['BTC_BB_Upper'], trading_results['BTC_BB_Lower'] = calculate_bollinger_bands(trading_results, 'BTC_Actual')
trading_results['SPY_BB_Mid'], trading_results['SPY_BB_Upper'], trading_results['SPY_BB_Lower'] = calculate_bollinger_bands(trading_results, 'SPY_Actual')

# Fill missing Bollinger Bands with the first valid value
trading_results[['BTC_BB_Mid', 'BTC_BB_Upper', 'BTC_BB_Lower']] = \
    trading_results[['BTC_BB_Mid', 'BTC_BB_Upper', 'BTC_BB_Lower']].fillna(method='bfill')

trading_results[['SPY_BB_Mid', 'SPY_BB_Upper', 'SPY_BB_Lower']] = \
    trading_results[['SPY_BB_Mid', 'SPY_BB_Upper', 'SPY_BB_Lower']].fillna(method='bfill')

# ============================================================
# Compute Predicted Returns from ML Model
# ============================================================
trading_results['BTC_Return'] = trading_results['BTC_Predicted'] / trading_results['BTC_PrevClose'] - 1
trading_results['SPY_Return'] = trading_results['SPY_Predicted'] / trading_results['SPY_PrevClose'] - 1

# ============================================================
# Adjust Risk-Adjusted Returns Using Annualized Volatility
# ============================================================
annualization_factor = np.sqrt(48 * 365)  # Convert 30-min volatility to annualized scale
trading_results['BTC_RiskAdj_Return'] = trading_results['BTC_Return'] / (trading_results['BTC_Volatility'] * annualization_factor)
trading_results['SPY_RiskAdj_Return'] = trading_results['SPY_Return'] / (trading_results['SPY_Volatility'] * annualization_factor)

# Fill missing values using previous values or a small default
trading_results['BTC_RiskAdj_Return'].fillna(method='bfill', inplace=True)
trading_results['SPY_RiskAdj_Return'].fillna(method='bfill', inplace=True)

# ============================================================
# Trading Signal Generation (Based on Multiple Market Signals)
# ============================================================
def generate_trading_signal(row):
    """Generate a combined signal based on ML predictions, Bollinger Bands, and other market signals."""
    if row['BTC_RiskAdj_Return'] > row['SPY_RiskAdj_Return'] and row['BTC_Actual'] < row['BTC_BB_Lower']:
        return 1  # Allocate to BTC
    if row['SPY_RiskAdj_Return'] > row['BTC_RiskAdj_Return'] and row['SPY_Actual'] < row['SPY_BB_Lower']:
        return 2  # Allocate to SPY
    return row.get('Signal', np.nan)  # Default to previous signal if no clear condition

# Ensure signals are only computed when necessary values exist
trading_results['Signal'] = trading_results.apply(
    lambda row: generate_trading_signal(row) if pd.notna(row['BTC_RiskAdj_Return']) and pd.notna(row['SPY_RiskAdj_Return']) else np.nan, axis=1
)

# Forward-fill missing signals to maintain previous positions
trading_results['Signal'].fillna(method='ffill', inplace=True)

# ============================
# Portfolio Initialization
# ============================
trading_results['BTC_Units'] = 0.0
trading_results['SPY_Units'] = 0.0
trading_results['TLT_Units'] = initial_investment * current_weights.get('TLT', 0) / initial_tlt_price
trading_results['Cash'] = initial_investment * current_weights.get('CASH', 0)
trading_results['Portfolio_Value'] = initial_investment

# Track position changes
trading_results['Position_Change'] = trading_results['Signal'].diff().abs()


C:\Users\jessi\AppData\Local\Temp\ipykernel_20872\1153125878.py:25: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  trading_results['SPY_Volatility'] = trading_results['SPY_Actual'].pct_change().rolling(1440).std()
C:\Users\jessi\AppData\Local\Temp\ipykernel_20872\1153125878.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tradin

### Trading simulation

In [31]:
# ============================
# Trading Simulation Loop (Incorporating Market Closures)
# ============================
for i in range(1, len(trading_results)):
    current_btc_price = trading_results['BTC_Actual'].iloc[i]
    current_spy_price = trading_results['SPY_Actual'].iloc[i] if not trading_results['SPY_Closed'].iloc[i] else trading_results['SPY_Actual'].iloc[i-1]
    current_tlt_price = trading_results['TLT_Close'].iloc[i] if not trading_results['TLT_Closed'].iloc[i] else trading_results['TLT_Close'].iloc[i-1]

    prev_btc = trading_results['BTC_Units'].iloc[i-1]
    prev_spy = trading_results['SPY_Units'].iloc[i-1]
    prev_tlt = trading_results['TLT_Units'].iloc[i-1]
    prev_cash = trading_results['Cash'].iloc[i-1]
    prev_signal = trading_results['Signal'].iloc[i-1]
    current_signal = trading_results['Signal'].iloc[i]

    spy_closed = trading_results['SPY_Closed'].iloc[i]
    tlt_closed = trading_results['TLT_Closed'].iloc[i]

    # Maintain minimum cash buffer
    btc_value = prev_btc * current_btc_price
    spy_value = prev_spy * current_spy_price
    tlt_value = prev_tlt * current_tlt_price
    total_value = btc_value + spy_value + tlt_value + prev_cash

    # Prevent trading SPY when market is closed
    if prev_signal != current_signal:
        if current_signal == 2 and spy_closed:
            trading_results.at[i, 'Signal'] = prev_signal  # Defer trade
        else:
            # Allocate new risk capital
            risk_capital = total_value * (1 - current_weights.get('CASH', 0))
            trading_results.at[i, 'BTC_Units'] = (risk_capital * current_weights.get('BTC', 0) * (1 - commission_rate)) / current_btc_price
            trading_results.at[i, 'SPY_Units'] = (risk_capital * current_weights.get('SPY', 0) * (1 - commission_rate)) / current_spy_price if not spy_closed else prev_spy

    # Compute final portfolio value
    final_value = btc_value + spy_value + tlt_value + trading_results.at[i, 'Cash']
    trading_results.at[i, 'Portfolio_Value'] = final_value


C:\Users\jessi\AppData\Local\Temp\ipykernel_20872\4180568970.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5436.699028403169' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trading_results.at[i, 'Portfolio_Value'] = final_value


### Buy and hold

In [32]:
#buy and hold strategy for spy
#Calculate buy & hold strategy for comparison
initial_spy = initial_investment / trading_results['SPY_Actual'].iloc[0]
trading_results['SPY_Buy_Hold_Value'] = initial_spy * trading_results['SPY_Actual']

initial_tlt = initial_investment / trading_results["TLT_Close"].iloc[0]
trading_results["TLT_Buy_Hold_Value"] = initial_tlt * trading_results["TLT_Close"]

# Calculate returns and metrics
total_return = (trading_results['Portfolio_Value'].iloc[-1] / initial_investment) - 1
num_trades = trading_results['Position_Change'].sum()

In [33]:
#check trading results
trading_results.head()

,Unnamed: 0,Unnamed: 0_x,Date,BTC_Actual,BTC_Predicted,BTC_PrevClose,BTC_BreakHigh2_lag1,Unnamed: 0_y,SPY_Actual,SPY_Predicted,...,SPY_RiskAdj_Return,Signal,BTC_Units,SPY_Units,TLT_Units,Cash,Portfolio_Value,Position_Change,SPY_Buy_Hold_Value,TLT_Buy_Hold_Value
0,0,17539,2025-01-01 00:00:00,94819.27,95000.13016,95282.72,False,3237.0,586.900,588.502670,...,-0.002596,NaN,0.000000,0.000000,0.0,2800.0,100000.000000,NaN,100000.000000,100000.000000
1,1,17540,2025-01-01 00:30:00,94687.97,94644.51788,94819.27,False,3238.0,587.060,586.968758,...,0.000788,NaN,0.004784,3.730087,0.0,2800.0,2800.000000,NaN,100027.261884,100005.709718
2,2,17541,2025-01-01 01:00:00,94358.80,94750.07166,94687.97,False,3239.0,585.860,587.495580,...,0.004989,NaN,0.009321,7.257463,0.0,2800.0,5436.699028,NaN,99822.797751,99948.612539
3,3,17542,2025-01-01 01:30:00,93961.69,94338.51126,94358.80,False,3240.0,587.205,585.988060,...,0.001470,NaN,0.013666,10.571425,0.0,2800.0,7937.429598,NaN,100051.967967,100045.677744
4,4,17543,2025-01-01 02:00:00,93836.11,93863.01374,93961.69,False,3241.0,584.960,586.748484,...,-0.005227,NaN,0.017699,13.725472,0.0,2800.0,10266.208327,NaN,99669.449651,99830.992349


In [34]:
#check the rows that display NA values in any column, and give me the column name that has NA values for each row
# Find rows with NA values and corresponding column names
rows_with_na = trading_results.isna().any(axis=1)  # Identify rows with NA values
na_columns_per_row = trading_results[rows_with_na].apply(lambda row: row.index[row.isna()].tolist(), axis=1)

# Convert to DataFrame for better readability
na_summary = pd.DataFrame({'Row Index': na_columns_per_row.index, 'NA Columns': na_columns_per_row.values})
na_summary.head()

,Row Index,NA Columns
0,0,"[SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa..."
1,1,"[SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa..."
2,2,"[SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa..."
3,3,"[SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa..."
4,4,"[SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa..."


In [35]:
print(na_summary["NA Columns"].unique)

<bound method Series.unique of 0       [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa...
1       [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa...
2       [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa...
3       [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa...
4       [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower, Signa...
                              ...                        
2827    [Unnamed: 0_y, SPY_Actual, SPY_Predicted, SPY_...
2828    [Unnamed: 0_y, SPY_Actual, SPY_Predicted, SPY_...
2829             [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower]
2830             [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower]
2831             [SPY_BB_Mid, SPY_BB_Upper, SPY_BB_Lower]
Name: NA Columns, Length: 2832, dtype: object>
